## Applied Data Science Capstone Assignment 2 :Segmenting and Clustering Neighborhoods in Toronto - Part 1


### For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

![alt text](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1555718400000&hmac=hY_G211KTmDqSbQCViiORKXZxJe2iUqeTTbg1hVMtY0)


Link for scrapping a table from a website: https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [1]:
# Import necessary libraries

import requests
import lxml.html as lh
import pandas as pd

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

This means that there are 3 columns per row

In [4]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


#### Creating Pandas DataFrame

Each header is appended to a tuple along with an empty list.

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

[288, 288, 288]

This shows that each of the 3 columns has exactly 288 rows

#### Creating the pandas data frame 

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

3.To create the above dataframe:

-  The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4.Submit a link to your Notebook on your Github repository. (10 marks)

- Note: There are different website scraping libraries and packages in Python. One of the most common packages is  BeautifulSoup. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

- The package is so popular that there is a plethora of tutorials and examples of how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

- Use the BeautifulSoup package or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe

In [18]:
# Access the top 5 rows of the data frame 
df.head()

,Postcode,Borough,Neighbourhood
0,Adelaide,Downtown Toronto,M5H
1,Agincourt,Scarborough,M1S
2,Agincourt North,Scarborough,M1V
3,Albion Gardens,Etobicoke,M9V
4,Alderwood,Etobicoke,M8W


#### Rearranging and renaming the columns

In [19]:
df.columns = ['Neighbourhood','Borough', 'Postcode']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head()

,Postcode,Neighbourhood,Borough
0,M5H,Adelaide,Downtown Toronto
1,M1S,Agincourt,Scarborough
2,M1V,Agincourt North,Scarborough
3,M9V,Albion Gardens,Etobicoke
4,M8W,Alderwood,Etobicoke


#### Cleaning the messy string in the Borough column

In [20]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Neighbourhood,Borough
0,M5H,Adelaide,Downtown Toronto
1,M1S,Agincourt,Scarborough
2,M1V,Agincourt North,Scarborough
3,M9V,Albion Gardens,Etobicoke
4,M8W,Alderwood,Etobicoke


#### Dropping all cells with a borough that is Not assigned

In [21]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Postcode,Neighbourhood,Borough
0,M5H,Adelaide,Downtown Toronto
1,M1S,Agincourt,Scarborough
2,M1V,Agincourt North,Scarborough
3,M9V,Albion Gardens,Etobicoke
4,M8W,Alderwood,Etobicoke
5,M3H,Bathurst Manor,North York
6,M5V,Bathurst Quay,Downtown Toronto
7,M2K,Bayview Village,North York
8,M9V,Beaumond Heights,Etobicoke
9,M5M,Bedford Park,North York


#### Combining Neighbourhoods based on similar Postcode and Borough  

In [22]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern ,Rouge"
1,M1C,Scarborough,"Highland Creek ,Port Union ,Rouge Hill"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park ,Ionview ,Kennedy Park"
7,M1L,Scarborough,"Clairlea ,Golden Mile ,Oakridge"
8,M1M,Scarborough,"Cliffcrest ,Cliffside ,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff ,Cliffside West"


#### Removing any space in the start of the string

In [23]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

#### Assigning Borough values to the Neignbourhood where vlaue is "Not assigned"

In [24]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [25]:
# Check if the Neighbourhood for Queen's Park changed 
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [26]:
# Check the shape of the data frame
df.shape

(103, 3)

#### Save this file to a csv

In [27]:
df.to_csv(r'df_toronto1.csv')